# Chunking for Video Script

In [21]:
import os
import boto3
from dotenv import load_dotenv
# Access the environment variables
load_dotenv('/Users/williamkapner/Documents/MSBA/GSB570AI/Code/.env')
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
discord_token = os.getenv("DISCORD_TOKEN")


boto3_bedrock = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

# ✅ Now you're safe to use this client in LangChain
embedding_model = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1"
)

In [22]:
import os
os.getcwd()
print(os.listdir())


['Video_Script_Chunking.ipynb', 'video_chunks.json', 'requiremnets.txt', 'video_script_to_db.ipynb', 'discordQASummerizer.ipynb', 'README.md', 'my_chunks.db', '.gitignore', 'formatted_chunks.json', '.git', 'load_data.py', 'CHAPTER 8 - Organic (MIKE).txt', 'script_ch8.json']


In [23]:
import boto3
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv
from langchain_community.chat_models import BedrockChat
import os

load_dotenv('/Users/williamkapner/Documents/MSBA/GSB570AI/Code/.env')
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")

# --------- 1. Setup AWS Credentials ---------
boto3_session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region  # or your Bedrock-supported region
)

# --------- 2. Load and Preprocess Your Script File ---------
# If your uploaded .ipynb is misnamed and actually a .csv or text:
try:
    df = pd.read_csv("CHAPTER 8 - Organic (MIKE).txt", engine="python")
    all_text = " ".join(df.astype(str).values.flatten())
except Exception:
    with open("CHAPTER 8 - Organic (MIKE).txt", "r") as f:
        all_text = f.read()

# --------- 3. Chunk the Text ---------
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_text(all_text)

# --------- 4. Titan Embeddings from Bedrock ---------
embeddings = BedrockEmbeddings(
    client=boto3_session.client("bedrock-runtime"),
    model_id="amazon.titan-embed-text-v1"
)

# --------- 5. Store in FAISS ---------
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore = FAISS.from_documents(documents, embedding=embeddings)

# --------- 6. Claude LLM from Bedrock ---------
llm = BedrockChat(
    client=boto3_session.client("bedrock-runtime"),
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.2, "max_tokens": 1024}
)

# --------- 7. Create RetrievalQA Chain ---------
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# --------- 8. Ask a Question ---------
question = "What makes a viral video convert?"
result = qa_chain({"query": question})

# --------- 9. Display Result ---------
print("\nAnswer:\n", result["result"])
print("\nRelevant Script Sections:\n")
for doc in result["source_documents"]:
    print(doc.page_content)


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            return content
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

: 

In [ ]:
import spacy

def fixed_size_chunking(text, chunk_size, overlap):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(text)
    tokens = [token.text for token in doc]

    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start += chunk_size - overlap  # move start forward with overlap

    return chunks

In [ ]:
def semantic_embedding_chunk(text, threshold):
    """
    Splits text into semantic chunks using sentence embeddings.
    Uses spaCy for sentence segmentation and SentenceTransformer for generating embeddings.

    :param text: The full text to chunk.
    :param threshold: Cosine similarity threshold for adding a sentence to the current chunk.
    :return: A list of semantic chunks (each as a string).
    """
    # Sentence segmentation
    #doc = nlp(text)
    #sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    sentences = fixed_size_chunking(text, 100, 10)

    chunks = []
    current_chunk_sentences = []
    current_chunk_embedding = None

    for sentence in sentences:
        # Generate embedding for the current sentence
        sentence_embedding = model.encode(sentence, convert_to_tensor=True)

        # If starting a new chunk, initialize it with the current sentence
        if current_chunk_embedding is None:
            current_chunk_sentences = [sentence]
            current_chunk_embedding = sentence_embedding
        else:
            # Compute cosine similarity between current sentence and the chunk embedding
            sim_score = util.cos_sim(sentence_embedding, current_chunk_embedding)
            if sim_score.item() >= threshold:
                # Add sentence to the current chunk and update the chunk's average embedding
                current_chunk_sentences.append(sentence)
                num_sents = len(current_chunk_sentences)
                current_chunk_embedding = ((current_chunk_embedding * (num_sents - 1)) + sentence_embedding) / num_sents
            else:
                # Finalize the current chunk and start a new one
                chunks.append(" ".join(current_chunk_sentences))
                current_chunk_sentences = [sentence]
                current_chunk_embedding = sentence_embedding

    # Append the final chunk if it exists
    if current_chunk_sentences:
        chunks.append(" ".join(current_chunk_sentences))

    return chunks

In [ ]:
def semantic_embedding_chunk(text, threshold):
    """
    Splits text into semantic chunks using sentence embeddings.
    Uses spaCy for sentence segmentation and SentenceTransformer for generating embeddings.

    :param text: The full text to chunk.
    :param threshold: Cosine similarity threshold for adding a sentence to the current chunk.
    :return: A list of semantic chunks (each as a string).
    """
    # Sentence segmentation
    #doc = nlp(text)
    #sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    sentences = fixed_size_chunking(text, 100, 10)

    chunks = []
    current_chunk_sentences = []
    current_chunk_embedding = None

    for sentence in sentences:
        # Generate embedding for the current sentence
        sentence_embedding = model.encode(sentence, convert_to_tensor=True)

        # If starting a new chunk, initialize it with the current sentence
        if current_chunk_embedding is None:
            current_chunk_sentences = [sentence]
            current_chunk_embedding = sentence_embedding
        else:
            # Compute cosine similarity between current sentence and the chunk embedding
            sim_score = util.cos_sim(sentence_embedding, current_chunk_embedding)
            if sim_score.item() >= threshold:
                # Add sentence to the current chunk and update the chunk's average embedding
                current_chunk_sentences.append(sentence)
                num_sents = len(current_chunk_sentences)
                current_chunk_embedding = ((current_chunk_embedding * (num_sents - 1)) + sentence_embedding) / num_sents
            else:
                # Finalize the current chunk and start a new one
                chunks.append(" ".join(current_chunk_sentences))
                current_chunk_sentences = [sentence]
                current_chunk_embedding = sentence_embedding

    # Append the final chunk if it exists
    if current_chunk_sentences:
        chunks.append(" ".join(current_chunk_sentences))

    return chunks

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util

file_path = "StudyDropshipping-ChatBot/CHAPTER 8 - Organic (MIKE).txt"
home_care_content = read_file(file_path)

nlp = spacy.load("en_core_web_md")
model = SentenceTransformer("all-MiniLM-L6-v2")

semantic_chunks = semantic_embedding_chunk(home_care_content, threshold=0.5)
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n{'-'*60}")

: 

In [ ]:
import re

file_path = "/Users/williamkapner/Downloads/CHAPTER 7 - Mindset.txt"

with open(file_path, "r", encoding="utf-8") as f:
    raw_script = f.read()

# Step 1: Normalize headers and section breaks
def preprocess_script(text):
    # Normalize line breaks
    text = re.sub(r'\n{2,}', '\n\n', text.strip())
    return text

# Step 2: Split into chunks by detecting headers (assuming each header is followed by a paragraph)
def chunk_script(text):
    chunks = []
    pattern = r'(?:^|\n{2,})([A-Z][A-Za-z0-9\s]+)\n+(.+?)(?=\n{2,}[A-Z]|\Z)'  # Header + Paragraph until next header or end
    matches = re.findall(pattern, text, flags=re.DOTALL)

    for i, (header, body) in enumerate(matches):
        chunk = {
            "chunk_id": i + 1,
            "header": header.strip(),
            "body": body.strip()
        }
        chunks.append(chunk)

    return chunks

# Step 3: Format for LLM ingestion or output as JSON/text
def print_chunks(chunks):
    for chunk in chunks:
        print(f"### Chunk {chunk['chunk_id']}: {chunk['header']}\n")
        print(chunk['body'])
        print("\n" + "-"*60 + "\n")

# Usage
processed = preprocess_script(raw_script)
chunked = chunk_script(processed)
print_chunks(chunked)


In [ ]:
import json

# Assume `chunked` is already created from earlier steps (a list of dictionaries)
# Each element looks like: { 'chunk_id': 1, 'header': ..., 'body': ... }

# Step 1: Convert to desired JSON format
formatted_chunks = []
for i, chunk in enumerate(chunked, start=1):
    formatted_chunk = {
        "id": i,
        "header": chunk["header"],
        "content": chunk["body"],
        "category": 'Organic Marketing',
        "source": 'course'
    }
    formatted_chunks.append(formatted_chunk)

# Step 2: (Optional) Save to JSON file
output_path = "formatted_chunks.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(formatted_chunks, f, indent=2, ensure_ascii=False)

# Step 3: (Optional) Print first few for verification
for fc in formatted_chunks[:5]:  # Just preview the first 3
    print(json.dumps(fc, indent=2))


In [ ]:
import json
import boto3

# Load your chunked JSON
with open("formatted_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# Construct a context string from all chunks
def build_context(chunks):
    context = ""
    for chunk in chunks:
        context += f"\n\n### {chunk['chunk_header']}\n{chunk['content']}\n"
    return context.strip()

# Ask a question using Claude via Bedrock
def ask_bedrock_claude(question, chunks, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
    context = build_context(chunks)

    prompt = f"""You are a helpful assistant. Use the CONTEXT below to answer the QUESTION at the end.

CONTEXT:
{context}

QUESTION: {question}
ANSWER:"""

    client = boto3.client("bedrock-runtime", region_name="us-west-2")

    body = {
        "prompt": prompt,
        "max_tokens": 500,
        "temperature": 0.2,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman", "\n\n"],
    }

    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body)
    )

    result = json.loads(response["body"].read())
    return result.get("completion", "").strip()

# Example usage
question = "What is the difference between organic and paid advertising?"
answer = ask_bedrock_claude(question, chunks)
print("\nAnswer:\n", answer)


In [ ]:
import os
import json
import boto3
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import BedrockEmbeddings
from langchain.schema import Document

# ✅ Step 1: Load chunks from JSON
with open("formatted_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ✅ Step 2: Convert to LangChain Documents
documents = [
    Document(
        page_content=chunk["content"],
        metadata={"chunk_header": chunk["chunk_header"], "paragraph_number": chunk["paragraph_number"]}
    )
    for chunk in chunks
]

# ✅ Step 3: Set up Bedrock Titan Embeddings with credentials
boto3_bedrock = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

embedding_model = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1"
)

# ✅ Step 4: Create and persist Chroma VectorStore
vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory="./dropshipping_vectorstore")
vectorstore.persist()

print("✅ Vector store created and saved.")


✅ Vector store created and saved.


/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/ipykernel_92395/4289400057.py:37: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
import json
import boto3

# Load your chunked JSON
with open("formatted_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# Construct a context string from all chunks
def build_context(chunks):
    context = ""
    for chunk in chunks:
        context += f"\n\n### {chunk['chunk_header']}\n{chunk['content']}\n"
    return context.strip()

# Ask a question using Claude via Bedrock
def ask_bedrock_claude(question, chunks, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
    context = build_context(chunks)

    prompt = f"""You are a helpful assistant. Use the CONTEXT below to answer the QUESTION at the end.

CONTEXT:
{context}

QUESTION: {question}
ANSWER:"""

    client = boto3.client("bedrock-runtime", region_name="us-west-2")

    body = {
        "prompt": prompt,
        "max_tokens": 500,
        "temperature": 0.2,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman", "\n\n"],
    }

    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body)
    )

    result = json.loads(response["body"].read())
    return result.get("completion", "").strip()

# Example usage
question = "What is the difference between organic and paid advertising?"
answer = ask_bedrock_claude(question, chunks)
print("\nAnswer:\n", answer)
